# text.FastWordpieceTokenizer


In [38]:
import tensorflow as tf
import tensorflow_text as tf_text 

In [4]:
vocab = ["they", "##'", "##re", "the", "great", "##est", "[UNK]","'", "re"]
tokenizer = tf_text.FastWordpieceTokenizer(vocab, token_out_type=tf.string)
tokens = [["they're the greatest", "the greatest"]]
tokenizer.tokenize(tokens)



2021-12-13 10:22:50.598706: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-13 10:22:50.685407: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-13 10:22:50.686429: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-13 10:22:50.690592: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

<tf.RaggedTensor [[[b'they', b"'", b're', b'the', b'great', b'##est'], [b'the', b'great', b'##est']]]>

# Load Tokenizer from hf.co

In [7]:
!wget https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt

--2021-12-13 10:26:24--  https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt
Resolving huggingface.co (huggingface.co)... 52.22.206.115, 34.204.221.201, 2600:1f18:147f:e800:33f3:2175:312f:eee6, ...
Connecting to huggingface.co (huggingface.co)|52.22.206.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 231508 (226K) [text/plain]
Saving to: ‘vocab.txt’

vocab.txt           100%[===================>] 226.08K  --.-KB/s    in 0.003s  

2021-12-13 10:26:24 (79.7 MB/s) - ‘vocab.txt’ saved [231508/231508]



In [21]:
with open("vocab.txt") as v:
    hf_vocab = [line.rstrip() for line in v.readlines()]

In [22]:
hf_vocab[1]

'[unused0]'

In [26]:
tokens = ["they're the greatest", "the greatest"]

In [61]:
tokenizer = tf_text.FastWordpieceTokenizer(hf_vocab, token_out_type=tf.int32)

_START_TOKEN = hf_vocab.index("[CLS]")
_END_TOKEN = hf_vocab.index("[SEP]")

def tokenize(input_text,max_length=16):
  # tokenize
  subtokens = tokenizer.tokenize(tokens)

  # add special tokens
  segments_combined, segment_ids = tf_text.combine_segments(
      [subtokens],
      start_of_sequence_id=_START_TOKEN,
      end_of_segment_id=_END_TOKEN)
  # pad
  padded_inputs = tf_text.pad_model_inputs(segments_combined, max_seq_length=16)
  return {"input_ids":padded_inputs[0],"attention_mask":padded_inputs[1]}

tf_res = tokenize(tokens)
tf_res

{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
 array([[ 101, 2027, 1005, 2128, 1996, 4602,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0],
        [ 101, 1996, 4602,  102,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0]], dtype=int32)>,
 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}

In [62]:
from transformers import AutoTokenizer

hf_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [63]:
hf_res = hf_tokenizer(tokens,return_tensors="tf",padding="max_length",max_length=16)
hf_res

{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[ 101, 2027, 1005, 2128, 1996, 4602,  102,    0,    0,    0,    0,
           0,    0,    0,    0,    0],
       [ 101, 1996, 4602,  102,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}

In [67]:
for key in list(tf_res.keys()):
  print(tf.equal(hf_res[key],tf_res[key]))

tf.Tensor(
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]], shape=(2, 16), dtype=bool)
tf.Tensor(
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]], shape=(2, 16), dtype=bool)
